In [6]:
!pip install pyttsx3
!pip install SpeechRecognition
!pip install pygame

jupyterlab 2.1.0 has requirement jupyterlab_server>=1.1.0, but you'll have jupyterlab-server 1.0.0 which is incompatible.
jupyterlab-server 1.0.0 has requirement jsonschema>=3.0.1, but you'll have jsonschema 2.6.0 which is incompatible.
You are using pip version 10.0.1, however version 20.3b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


jupyterlab 2.1.0 has requirement jupyterlab_server>=1.1.0, but you'll have jupyterlab-server 1.0.0 which is incompatible.
jupyterlab-server 1.0.0 has requirement jsonschema>=3.0.1, but you'll have jsonschema 2.6.0 which is incompatible.
You are using pip version 10.0.1, however version 20.3b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


jupyterlab 2.1.0 has requirement jupyterlab_server>=1.1.0, but you'll have jupyterlab-server 1.0.0 which is incompatible.
jupyterlab-server 1.0.0 has requirement jsonschema>=3.0.1, but you'll have jsonschema 2.6.0 which is incompatible.
You are using pip version 10.0.1, however version 20.3b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
# define rooms and items

# Furniture Elements

couch = {
    "name": "couch",
    "type": "furniture"
}

piano = {
    "name": "piano",
    "type": "furniture"
}

king_bed = {
    "name": "king bed",
    "type": "furniture"
}

table = {
    "name": "table",
    "type": "furniture"
}

piano = {
    "name": "piano",
    "type": "furniture"
}

# Doors

gate = {
    "name": "gate",
    "type": "door"
}

door = {
    "name": "door",
    "type": "door"
}

exit = {
    "name": "exit",
    "type": "door"
}

# Keys

key_a = {
    "name": "key for gate",
    "type": "key",
    "target": gate
}

key_c = {
    "name": "key for door",
    "type": "key",
    "target": door
}

key_d = {
    "name": "key for exit",
    "type": "key",
    "target": exit
}

#Rooms
 
game_room = {
    "name": "game room",
    "type": "room"
}

bedroom_1 = {
    "name": "bedroom 1",
    "type": "room"
}

living_room = {
    "name": "living room",
    "type": "room"
}

outside = {
  "name": "outside"
}
 
all_rooms = [game_room, bedroom_1, living_room, outside] # removed bedroom_2
 
all_doors = [gate, door, exit] # removed access, door for bedroom_2

all_keys = [key_a, key_c, key_d] # removed key_b, door for bedroom_2
 
# define which items/rooms are related
 
object_relations = {
    "game room": [couch, piano, gate],
    "bedroom 1": [king_bed, gate, door], # removed access, door for bedroom_2
    #"bedroom 2": [double_bed, dresser, access],
    "living room": [table, door, exit],
    "piano": [key_a],
    #"double bed": [key_c],
    "king bed": [key_c], #replaced b with c
    "table": [key_d],
    "outside": [exit],
    "gate": [game_room, bedroom_1],
    #"access": [bedroom_1, bedroom_2],
    "door": [bedroom_1, living_room],
    "exit": [outside]
}
 
# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.
 
INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

In [4]:
###IMPORTAÇÃO DE PACOTES
import pyttsx3
import speech_recognition as sr
import pygame
from pygame import mixer

###DEFINIR FUNCTIONS PARA SOM DA PORTA
def doorsound():
    mixer.init()
    a = mixer.Sound('scary_door.wav')
    b = mixer.Sound.play(a)
    while b.get_busy():
        pygame.time.delay(10)

###DEFINIR FUNCTIONS PARA SOM QUANDO CHEGAMOS AO TARGET ROOM=OUTSIDE#####      
def escaped():
    mixer.init()
    a = mixer.Sound('you_win.wav')
    b = mixer.Sound.play(a)
    while b.get_busy():
        pygame.time.delay(10)

###########################################
#COLORAMA
from colorama import Fore, Back, Style  

###########################################

def linebreak():
    """
    Print a line break
    """
    print("\n\n")
def start_game():
    """
    Start the game
    """
#Text to speech called via engine
    
    engine = pyttsx3.init() 
    engine.say("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!") #Engine receives the text
    engine.runAndWait() 
    engine.stop() 
    
    print(Fore.RED + "You wake up on a couch, and find yourself in a strange house, with no windows, which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!")
    
    play_room(game_state["current_room"])
    
def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the room!")
        escaped()
        engine = pyttsx3.init() 
        engine.say("Congrats! You escaped the room! Mind the gap, there is a cliff")
        engine.runAndWait()
        engine.stop()
        
        print("Wanna play again? Yes or No")
        r = sr.Recognizer()
        with sr.Microphone() as source:
            engine = pyttsx3.init() 
            engine.say("Wanna play again? Yes or No")
            engine.runAndWait()
            engine.stop() 
            r.adjust_for_ambient_noise(source)
            data = None
            while data is None:    
                try:
                    data = r.record(source, duration=3)
                    intended_action = r.recognize_google(data,language="en-US")
                    intended_action = str(intended_action.replace(" ", "").lower())
                    print(intended_action)
                        
                except:
                    data = None
                    print('Please, repeat...')
                    engine = pyttsx3.init()
                    engine.say('Please, repeat...')
                    engine.runAndWait()
                    engine.stop()
        
        if intended_action == "yes":
            engine = pyttsx3.init()
            engine.say("Just kidding, this doesn't work. Did you pay attention to the last slide of Group One presentation?")
            engine.runAndWait()
            engine.stop()
            exit()
 
        if intended_action == "no":
            exit()
        
        else:
            print('Please, repeat...')
            engine = pyttsx3.init()
            engine.say('Please, repeat...')
            engine.runAndWait()
            engine.stop()
            
    else:
        print("You are now in " + room["name"])
        
        #Text to speech called via engine
        engine = pyttsx3.init()
        engine.say("You are now in " + room["name"])
        engine.runAndWait()
        engine.stop()      
        
        r = sr.Recognizer()
        with sr.Microphone() as source:
            engine = pyttsx3.init() #Speech to text init with recognizer
            engine.say("What would you like to do? See or Go?")     
            r.adjust_for_ambient_noise(source)
            engine.say("Speak Now!")
            engine.runAndWait()
            engine.stop()   
            #Speech to text
            data = None
            while data is None:
                try:
                    data = r.record(source, duration=3)
                    intended_action = r.recognize_google(data,language="en-US")
                    intended_action = str(intended_action.replace(" ", "").lower())
                    print(intended_action)
                except:
                    data = None
                    print('Please, repeat...')
                    engine = pyttsx3.init()
                    engine.say('Please, repeat...')
                    engine.runAndWait()
                    engine.stop()
                          
        if "see" == intended_action:
            explore_room(room)
            play_room(room)
            
        elif "go" == intended_action:          
            r = sr.Recognizer()
            with sr.Microphone() as source:
                engine = pyttsx3.init()
                engine.say("Where do you wanna go?") 
                r.adjust_for_ambient_noise(source)
                engine.say("Speak Now!")  
                engine.runAndWait()
                engine.stop() 
                data = None
                while data is None:    
                    try:
                        data = r.record(source, duration=3)
                        intended_action = r.recognize_google(data,language="en-US")
                        intended_action = str(intended_action.replace(" ", "").lower())
                        print(intended_action)
                        
                    except:
                        data = None
                        print('Please, repeat...')
                        engine = pyttsx3.init()
                        engine.say('Please, repeat...')
                        engine.runAndWait()
                        engine.stop()
                        
            examine_item(intended_action)
            play_room(room)
            
        else:
            print("In English Please! Not sure what you mean. Say See or Go.")
            engine = pyttsx3.init()
            engine.say("In English Please! Not sure what you mean. Say See or Go.")
            engine.runAndWait()
            engine.stop()
            play_room(room)
        linebreak()
               
def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("Exploring the " + room["name"] + ". You find " + ", ".join(items))
    
    engine = pyttsx3.init()
    engine.say("Exploring the " + room["name"] + ". You find " + ", ".join(items))
    engine.runAndWait()
    engine.stop()
    
def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room
        
def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    
    for item in object_relations[current_room['name']]:
        if(item["name"].replace(" ", "") == item_name):
            output = "You examine " + item_name + ". "
            
            engine = pyttsx3.init()
            engine.say("You examine " + item_name) 
            engine.runAndWait()
            engine.stop()
            
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                
                if(have_key):
                    output += "You unlock it with a key you have. "
                    engine = pyttsx3.init()
                    engine.say(str(output))
                    engine.runAndWait()
                    engine.stop()
                    next_room = get_next_room_of_door(item, current_room)
                    
                else:
                    output += "How silly you are? It's locked, you don't have the key."
                    engine = pyttsx3.init()
                    engine.say("How silly you are? It's locked, you don't have the key.")
                    engine.runAndWait()
                    engine.stop()                  
                    
            else:
                
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + ". "
                    engine = pyttsx3.init()
                    engine.say(str(output))
                    engine.runAndWait()
                    engine.stop()
                
                else:
                    output += " Ah , Ah , Ah! You nailed it! There isn't anything interesting about it."
                    engine = pyttsx3.init()
                    engine.say(str(output))
                    engine.runAndWait()
                    engine.stop()
            print(output)
            break
            
    if(output is None):
        print("Come on, you can do better than this! Please request only items available in the current room.")
        
        engine = pyttsx3.init()
        engine.say("Come on! You can do better than this! Please request only items available in the current room.")
        engine.runAndWait()
        engine.stop()
        
    if(next_room and have_key == True):     
        doorsound()
        play_room(next_room)
        
    else:
        play_room(current_room)


In [ ]:
game_state = INIT_GAME_STATE.copy()

start_game()


You wake up on a couch, and find yourself in a strange house, with no windows, which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!
You are now in game room
esternocleidomastoideo
In English Please! Not sure what you mean. Say See or Go.
You are now in game room
see
Exploring the game room. You find couch, piano, gate
You are now in game room
go
piano
You examine piano. You find key for gate. 
You are now in game room
go
gate
You examine gate. You unlock it with a key you have. 
You are now in bedroom 1
see
Exploring the bedroom 1. You find king bed, gate, door
You are now in bedroom 1
go
kingboo
Come on, you can do better than this! Please request only items available in the current room.
You are now in bedroom 1
kingbedgo
In English Please! Not sure what you mean. Say See or Go.
You are now in bedroom 1
go
kingboo
Come on, you can do better than this! 